In [26]:
import pandas as pd
import numpy as np
import pickle
import keras
import cv2
from keras.models import Model,Sequential,model_from_json
from keras.layers import *
from keras import optimizers
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.utils import np_utils
from keras import backend as K


In [13]:
training=pd.read_csv('train_final.csv',index_col=False)
labels=training[['784']]

In [14]:
labels

,784
0,10
1,10
2,10
3,10
4,10
...,...
15255,14
15256,14
15257,14
15258,14


In [15]:
training.drop(training.columns[[784]],axis=1,inplace=True)

In [16]:
type(labels)

pandas.core.frame.DataFrame

In [17]:
labels=np.array(labels)

In [18]:
labels.shape

(15260, 1)

In [19]:
category=np_utils.to_categorical(labels,num_classes=15)
print(category[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [20]:
resize=[]
for i in range(15260):
    resize.append(np.array(training[i:i+1]).reshape(1,28,28))

In [95]:
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=(1,28,28), data_format="channels_first",activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(15, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [96]:
model.fit(np.array(resize), category, epochs=38, batch_size=100,shuffle=True,verbose=1)

Epoch 1/38
153/153 [==============================] - 1s 4ms/step - loss: 2.4446 - accuracy: 0.3434
Epoch 2/38
153/153 [==============================] - 1s 4ms/step - loss: 1.0620 - accuracy: 0.6543
Epoch 3/38
153/153 [==============================] - 1s 4ms/step - loss: 0.6576 - accuracy: 0.7804
Epoch 4/38
153/153 [==============================] - 1s 4ms/step - loss: 0.4808 - accuracy: 0.8315
Epoch 5/38
153/153 [==============================] - 1s 4ms/step - loss: 0.3883 - accuracy: 0.8632
Epoch 6/38
153/153 [==============================] - 1s 4ms/step - loss: 0.3357 - accuracy: 0.8771
Epoch 7/38
153/153 [==============================] - 1s 4ms/step - loss: 0.2979 - accuracy: 0.8911
Epoch 8/38
153/153 [==============================] - 1s 4ms/step - loss: 0.2670 - accuracy: 0.9005
Epoch 9/38
153/153 [==============================] - 1s 4ms/step - loss: 0.2388 - accuracy: 0.9108
Epoch 10/38
153/153 [==============================] - 1s 4ms/step - loss: 0.2274 - accuracy: 0.9102

In [116]:
img = cv2.imread('fiveplusone.jpg',cv2.IMREAD_GRAYSCALE)
cv2.waitKey(0)
if img is not None:
    img=~img
    ret,thresh=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    ctrs,ret=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnt=sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
    w=int(28)
    h=int(28)
    train_data=[]
    #print(len(cnt))
    rects=[]
    for c in cnt :
        x,y,w,h= cv2.boundingRect(c)
        rect=[x,y,w,h]
        rects.append(rect)
    #print(rects)
    bool_rect=[]
    for r in rects:
        l=[]
        for rec in rects:
            flag=0
            if rec!=r:
                if r[0]<(rec[0]+rec[2]+10) and rec[0]<(r[0]+r[2]+10) and r[1]<(rec[1]+rec[3]+10) and rec[1]<(r[1]+r[3]+10):
                    flag=1
                l.append(flag)
            if rec==r:
                l.append(0)
        bool_rect.append(l)
    #print(bool_rect)
    dump_rect=[]
    for i in range(0,len(cnt)):
        for j in range(0,len(cnt)):
            if bool_rect[i][j]==1:
                area1=rects[i][2]*rects[i][3]
                area2=rects[j][2]*rects[j][3]
                if(area1==min(area1,area2)):
                    dump_rect.append(rects[i])
    #print(len(dump_rect)) 
    final_rect=[i for i in rects if i not in dump_rect]
    #print(final_rect)
    for r in final_rect:
        x=r[0]
        y=r[1]
        w=r[2]
        h=r[3]
        im_crop =thresh[y:y+h+10,x:x+w+10]
        

        im_resize = cv2.resize(im_crop,(28,28))
        #cv2.imshow("work",im_resize)
        cv2.waitKey(0)
        #cv2.destroyAllWindows()

        im_resize=np.reshape(im_resize,(1,28,28))
        train_data.append(im_resize)

In [117]:
s=''
for i in range(len(train_data)):
    train_data[i]=np.array(train_data[i])
    train_data[i]=train_data[i].reshape(1,1,28,28)
    result=model.predict_classes(train_data[i])
    if(result[0]==10):
        s=s+'-'
    if(result[0]==11):
        s=s+'+'
    if(result[0]==12):
        s=s+'*'
    if(result[0]==13):
      s=s+'/'
    if(result[0]==14):
      s=s+'='
    if(result[0]==0):
        s=s+'0'
    if(result[0]==1):
        s=s+'1'
    if(result[0]==2):
        s=s+'2'
    if(result[0]==3):
        s=s+'3'
    if(result[0]==4):
        s=s+'4'
    if(result[0]==5):
        s=s+'5'
    if(result[0]==6):
        s=s+'6'
    if(result[0]==7):
        s=s+'7'
    if(result[0]==8):
        s=s+'8'
    if(result[0]==9):
        s=s+'9'
    
print(s)

3=+9


In [106]:
res=None
if(s[-2]=='='):
  res=s[-1]
  s=s[:-2]
predicted=eval(s)
if res is None:
  print("Answer to the Question is:",str(predicted))
elif(res==str(predicted)):
  print("Correct Answer")
else:
  print("Wrong Answer")


Answer to the Question is: 13
